# Introduction

The provided notebook is a Python program for cryptocurrency data analysis and portfolio optimization. It uses historical price data and market cap data for selected cryptocurrencies to perform portfolio optimization using the Efficient Frontier approach. The script aims to calculate the profit/loss of the portfolio over different time periods and with varying numbers of coins.

# Notebook
## Import libs

In [1]:
import sys
sys.path.append("../")

import itertools
from time import time

import numpy as np
import pandas as pd


from src import cryptorama as cc

## Functions

In [2]:
def calculate_profit(df, _top_100, _n_coins, _mu_method, _cov_method, _obj_function, _budget, _n_days_vector, sell_day, _drop,
                     _hodl, _save_dir):

    crypto_class = cc.CryptoPortfolio(top_100, _budget, _n_coins, _hodl, _save_dir)
    results = {}
    portf = {}

    for n_days in _n_days_vector[:-1]:
        crypto_class.validate_from_past_specific_dates(_n_coins, float(n_days), sell_day, _mu_method, _cov_method, _obj_function, _drop)
        date = df.iloc[n_days].name

        results[date] = crypto_class.p_l_specific
        portf[date] = crypto_class.portfolio_from_past_specific

    p_l = sum(results.values())

    return p_l, results, portf


def check_coins(portfolio):
    # Convert the dictionary to a list of DataFrames
    dfs = [pd.DataFrame(portfolio[date]) for date in portfolio]

    # Concatenate the list of DataFrames into a single DataFrame
    result_df = pd.concat(dfs, keys=portfolio.keys())

    def get_total_coins(data):
        print(data["Coin"].iloc[0], data["n_coins"].sum())

    result_df.groupby("Coin").apply(get_total_coins)

## Inputs

In [4]:
start = time()

top_100 = False
drop = False
hodl = True

save_dir = './legacy_data'
buy_year = 3
sell_year = 1
sell_day = int(365 * sell_year)
n_days_vector = np.arange(int(365 * buy_year), int(365 * sell_year), -30)
df_mc = pd.read_csv('./legacy_data/All_cryptos_market_cap.csv', index_col=0)


## Optimization parameters

In [5]:
possible_mu_methods = ["mean", "exp", "capm"]
possible_cov_methods = ["sample", "exp"]
possible_obj_functions = ["quadratic", "sharpe", "min_volat"]

# Get all combinations of mu_method, cov_method, and obj_function
all_combinations = list(itertools.product(possible_mu_methods, possible_cov_methods, possible_obj_functions))

## Variables initialization

In [6]:
# you place 100e each month per category for the past n years
budget = 100

maxpl2 = 0
maxpl10 = 0
maxpl20 = 0


best_portf_2 = None
best_portf_10 = None
best_portf_20 = None

best_res_2 = None
best_res_10 = None 
best_res_20 = None  

optimized_variables = dict()

## Run Analysis

In [8]:
# Test the portfolio_optimization function with all combinations
start = time()
for mu_method, cov_method, obj_function in all_combinations:

    n_coins = 2
    print(f'\n===== {n_coins} COINS ======\n')
    
#     try:
    p_l_2, results_2, portf_2 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                    obj_function, budget, n_days_vector, drop, hodl, save_dir)
#     except Exception as e:
#         pass
    if p_l_2 >= maxpl2:
        maxpl2 = p_l_2
        best_portf_2 = portf_2
        best_res2 = results_2
        
        optimized_variables["2_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
        
    n_coins = 10
    print(f'\n===== {n_coins} COINS ======\n')
    
    try:
        p_l_10, results_10, portf_10 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                        obj_function, budget, n_days_vector, drop, hodl, save_dir)
    except Exception as e:
        pass
    if p_l_10 >= maxpl10:
        maxpl10 = p_l_10
        best_portf_10 = portf_10
        best_res10 = results_10
        
        optimized_variables["10_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
            
    n_coins = 20

    print(f'\n===== {n_coins} COINS ======\n')

    try:
        p_l_20, results_20, portf_20 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                        obj_function, budget, n_days_vector, drop, hodl, save_dir)
    except Exception as e:
        pass
    
    if p_l_20 >= maxpl20:
        maxpl20 = p_l_20
        best_portf_20 = portf_20
        best_res20 = results_20
        
        optimized_variables["20_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
        
        

end = time()
print(f'Time elapsed: {(end - start) / 60}')


===== 2 COINS ======



TypeError: calculate_profit() missing 1 required positional argument: '_save_dir'

In [ ]:
check_coins(best_portf_20)
print(maxpl20)
optimized_variables["20_coins"]

In [ ]:
check_coins(best_portf_2)
print(maxpl2)
optimized_variables["2_coins"]

In [ ]:
check_coins(best_portf_10)
print(maxpl10)
optimized_variables["10_coins"]

## Results

In [ ]:
print(f'Profit 2c: {round(maxpl2, 2)} $')
print(f'Profit 10c: {round(maxpl10, 2)} $')
print(f'Profit 20c: {round(maxpl20, 2)} $')
# print(f'Profit 100c: {round(maxpl100, 2)} $')

print(f'\nInvestment: {3 * budget * len(n_days_vector)} $')
print(f'Final Profit: {round(maxpl2 + maxpl10 + maxpl20 , 2)} $') # + maxpl100
print(f'Final Profit: {round((maxpl2 + maxpl10 + maxpl20 ) / (3 * budget * len(n_days_vector)) * 100, 2)} %\n') # + maxpl100

end = time()
print(f'Time elapsed: {(end - start) / 60}')

# Conclusion

## 2 Coins

In [ ]:
optimized_variables["2_coins"]

## 10 Coins

In [ ]:
optimized_variables["10_coins"]

## 20 Coins

In [ ]:
optimized_variables["20_coins"]